# CS109b Final Project

## Milestone2

by Danqing Wang, Wenshan Zheng, Zecai Liang

----------

In [12]:
# import libraries
import numpy as np 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import random

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import urllib
from bs4 import BeautifulSoup
import time
import re
import tmdbsimple as tmdb
tmdb.API_KEY = '4074d0170761c40d9c07d9016ddd4965'
from collections import Counter

from imdb import IMDb
ia = IMDb()

import sys


import warnings
warnings.filterwarnings('ignore')

---
## Part1. Download Data

For demenstration of code: we use 100 movies

### 1.1 Download Data from TMDB

 #### 1.1.1 Get list of IMDB and TMDB IDs 

In [13]:
# set up some basic url link strings, to be used later
APIKeyZ = "api_key=4074d0170761c40d9c07d9016ddd4965"
base_url_search = "https://api.themoviedb.org/3/discover/movie?"
popular_desc = "&sort_by=popularity.desc"
year =  "&primary_release_year={}"
page_number = '&page={}'
query_url = 'https://api.themoviedb.org/3/movie/{}?'

In [14]:
# query latest movie and get the largest ID from TMDB
latest_url =  query_url.format("latest") + APIKeyZ
page = urllib.urlopen(latest_url).read()
soup = BeautifulSoup(page, "lxml")
prettified = soup.prettify()
movie_index = prettified.find(',"id"')
j_beginning = 6
movie_id_latest = ''
while (prettified[movie_index + j_beginning].isdigit()):
    movie_id_latest += str(prettified[movie_index + j_beginning])
    j_beginning += 1
movie_id_latest = int(movie_id_latest)

In [15]:
# randomly generate 200K movies ids without replacement to get movie TMDB IDs (it's possible that some of these IDs return empty)
movie_sample = 200000
movie_list = np.random.choice(range(1, movie_id_latest), movie_sample, replace = False)

In [16]:
# get movies' IMDB and TMDB ID, to be used for further search of IMDB ID
time.sleep(10)
IMDB_id_list = []
TMDB_id_list = []

for i in range(len(movie_list)):
    movie_url =  query_url.format(movie_list[i]) + APIKeyZ
    page = urllib.urlopen(movie_url).read()
    soup = BeautifulSoup(page, "lxml")
    prettified = soup.prettify()
    movie_index = prettified.find('"imdb_id"')
    if movie_index != -1:      
        j_beginning = 13
        movie_id_temp = ''
        while (prettified[movie_index + j_beginning].isdigit()):
            movie_id_temp += str(prettified[prettified.find('"imdb_id"') + j_beginning])
            j_beginning += 1
        if len(movie_id_temp) == 7:
            IMDB_id_list += [movie_id_temp]
            TMDB_id_list += [movie_list[i]]
            
    if len(IMDB_id_list) >= 100: # we use 25K for real data construction
        break    
    if i % 40 == 39:
        time.sleep(11)

In [17]:
# Write out movie TMDB IDs
thefile = open('tmdb_id_demo.txt', 'w')
for item in TMDB_id_list:
    thefile.write("%s\n" % item)
thefile.close()

In [18]:
# Write out movie IMDB IDs
thefile = open('imdb_id_demo.txt', 'w')
for item in IMDB_id_list:
    thefile.write("%s\n" % item)
thefile.close()

#### 1.1.2 Read in TMDB ID and get all TMDB information

In [19]:
text_file = open('tmdb_id_demo.txt', 'r')
tmdb_id_list = text_file.read().split('\n')
text_file.close()

In [20]:
# search one movie by ID and create list of column names
movie = tmdb.Movies(tmdb_id_list[0])
response = movie.info()
columns = []
for i in range(len(response)):
    column_temp = [str(response.items()[i][0])]
    columns += column_temp

In [21]:
# create a dataframe to store information of top 500 popular movies
index = range(len(tmdb_id_list)) 
df1 = pd.DataFrame(index = index, columns=columns)
df1 = df1.fillna(0)

In [22]:
# store information into dataframe, some information received is stroed as a list thus we convert it into strings
time.sleep(10)
for i in range(len(index) - 1):    
    movie = tmdb.Movies(tmdb_id_list[i])
    response = movie.info()
    if i % 40 == 39:
        time.sleep(11)
    for j in range(0, len(columns)):
        if j == 1:
            country_temp = []
            for k in range(len(response[columns[j]])):
                country_temp += [(response[columns[j]][k]['name'])]
            df1.iloc[i,j] = str(country_temp)
        elif j == 6:
            genre_temp = []
            for k in range(len(response[columns[j]])):
                genre_temp += [(response[columns[j]][k]['name'])]
            df1.iloc[i,j] = str(genre_temp)
        elif j == 11:
            if response[columns[j]] != None:
                df1.iloc[i,j] = str(response[columns[j]])
        elif j == 14:
            language_temp = []
            for k in range(len(response[columns[j]])):
                language_temp += [(response[columns[j]][k]['iso_639_1'])]
            df1.iloc[i,j] = str(language_temp)
        elif j == 18:
            company_temp = []
            for k in range(len(response[columns[j]])):
                company_temp += [(response[columns[j]][k]['name'])]
            df1.iloc[i,j] = str(company_temp)
        else:
            df1.iloc[i,j] = response[columns[j]]
    if i % 1000 ==0:
        print i
df1.head()

0


,poster_path,production_countries,revenue,overview,video,id,genres,title,tagline,vote_count,...,imdb_id,adult,backdrop_path,production_companies,release_date,popularity,original_title,budget,vote_average,runtime
0,/4NF4TOuo8fSTlIebXB0ANQWU9DU.jpg,"[u'United States of America', u'United Kingdom']",0,Martin Scorsese celebrates American movies fro...,False,55676,[u'Documentary'],A Personal Journey with Martin Scorsese Throug...,,25,...,tt0112120,False,/sOc3h8LtxCN6L6IwLICfDgXfLtG.jpg,"[u'Miramax Films', u'British Film Institute (B...",1995-05-21,0.147509,A Personal Journey with Martin Scorsese Throug...,0,8.2,225.0
1,/uxjCR9YDvYJhYDrZ20IXYwUUums.jpg,[],0,The Idol is a 1966 British drama film directed...,False,278777,[],The Idol,,0,...,tt0060533,False,None,[],1966-02-01,0.013512,The Idol,0,0.0,109.0
2,None,[u'Austria'],0,None,False,269376,"[u'Drama', u'Comedy']",Biedermann und die Brandstifter,None,0,...,tt1291512,False,None,[u'ORF'],1963-05-10,0.000513,Biedermann und die Brandstifter,0,0.0,NaN
3,/nFRjlQKZKkbMhenK9gNdmM61yey.jpg,[u'Japan'],0,A film adaptation of the Last Nanto General st...,False,114372,"[u'Action', u'Animation']",Fist of the North Star: Legend of Raoh - Chapt...,,9,...,tt1135522,False,/dl1uxhH60OxxH0m6LWoGZpr6BMV.jpg,[u'TMS'],2007-04-28,0.191590,真救世主伝説 北斗の拳 ラオウ伝 激闘の章,0,5.7,95.0
4,/UmksKAlD51DKiHvvwfcshqTyXz.jpg,[],0,"Twenty years ago, Jacob and his classmates col...",False,428307,[u'Documentary'],I Bought a Rain Forest,,0,...,tt1588874,False,None,[],2010-01-30,0.000715,I Bought a Rain Forest,0,0.0,58.0


In [23]:
# write the dataframe out
df1.to_csv('TMDb_data_demo.csv', encoding='utf-8')

#### 1.1.3 Download posters

In [24]:
# read in TMDB dataframe
df1 = pd.read_csv('TMDb_data_demo.csv',index_col=0)
df1

,poster_path,production_countries,revenue,overview,video,id,genres,title,tagline,vote_count,...,imdb_id,adult,backdrop_path,production_companies,release_date,popularity,original_title,budget,vote_average,runtime
0,/4NF4TOuo8fSTlIebXB0ANQWU9DU.jpg,"[u'United States of America', u'United Kingdom']",0,Martin Scorsese celebrates American movies fro...,False,55676,[u'Documentary'],A Personal Journey with Martin Scorsese Throug...,NaN,25,...,tt0112120,False,/sOc3h8LtxCN6L6IwLICfDgXfLtG.jpg,"[u'Miramax Films', u'British Film Institute (B...",1995-05-21,0.147509,A Personal Journey with Martin Scorsese Throug...,0,8.2,225.0
1,/uxjCR9YDvYJhYDrZ20IXYwUUums.jpg,[],0,The Idol is a 1966 British drama film directed...,False,278777,[],The Idol,NaN,0,...,tt0060533,False,NaN,[],1966-02-01,0.013512,The Idol,0,0.0,109.0
2,NaN,[u'Austria'],0,NaN,False,269376,"[u'Drama', u'Comedy']",Biedermann und die Brandstifter,NaN,0,...,tt1291512,False,NaN,[u'ORF'],1963-05-10,0.000513,Biedermann und die Brandstifter,0,0.0,NaN
3,/nFRjlQKZKkbMhenK9gNdmM61yey.jpg,[u'Japan'],0,A film adaptation of the Last Nanto General st...,False,114372,"[u'Action', u'Animation']",Fist of the North Star: Legend of Raoh - Chapt...,NaN,9,...,tt1135522,False,/dl1uxhH60OxxH0m6LWoGZpr6BMV.jpg,[u'TMS'],2007-04-28,0.191590,真救世主伝説 北斗の拳 ラオウ伝 激闘の章,0,5.7,95.0
4,/UmksKAlD51DKiHvvwfcshqTyXz.jpg,[],0,"Twenty years ago, Jacob and his classmates col...",False,428307,[u'Documentary'],I Bought a Rain Forest,NaN,0,...,tt1588874,False,NaN,[],2010-01-30,0.000715,I Bought a Rain Forest,0,0.0,58.0
5,/lHWzF3iuL28qFnS2fK9WEaId8rv.jpg,[u'South Korea'],0,Love Exposure explores the very roots of femal...,False,48411,"[u'Action', u'Comedy', u'Drama', u'Romance']",Love Exposure,"These days, women want it all: good job, hot r...",1,...,tt1201143,False,NaN,[u'Sidus Pictures'],2007-10-18,0.024656,어깨너머의 연인,0,3.0,100.0
6,/kN25TXLAnmbvO8a8EpTokRqFozZ.jpg,"[u'France', u'Germany', u'United Kingdom']",0,Berlin in June of 1940. While Nazi propaganda ...,False,334522,[u'Drama'],Alone in Berlin,NaN,20,...,tt3026488,False,/5aJUUoA4i80gIXp1ohPg9ehS1IF.jpg,"[u'X-Filme Creative Pool', u'Master Movies', u...",2016-10-13,1.179829,Alone in Berlin,0,6.3,103.0
7,/mHUTz2wP7bA9lwieiqbNVYS2iyr.jpg,[],0,"Two close friends from Sudbury, Ontario take a...",False,40928,[u'Drama'],Half a Person,NaN,2,...,tt1089016,False,NaN,[],2008-01-01,0.000979,Half a Person,0,2.5,0.0
8,/2FajUVgfMuUZF2Cbub9md5hV4ib.jpg,[],0,young prince is taken for tuition at a seaside...,False,153272,[],Taxandria,NaN,3,...,tt0117865,False,/vDbrDKMtbKAn8relSco5blIDeq1.jpg,[],1994-01-01,0.000593,Taxandria,0,6.3,82.0
9,/xwmQSREfUZYh4fIOtdk1Ufmieka.jpg,[],0,An angry child.,False,362134,"[u'Horror', u'Comedy', u'Drama']",My Johnny,NaN,0,...,tt3701250,False,NaN,[],2014-04-20,0.005192,My Johnny,0,0.0,30.0


In [25]:
import urllib
path_poster = "http://image.tmdb.org/t/p/w500/{}"
path_poster_local = '{}.jpg'
poster_list = []

for i in range(len(df1)):
    extend_path = df1.loc[i, "poster_path"]
    if len(str(extend_path)) > 4:
        poster_list += [i]
        IMDB_ID = df1.loc[i, "imdb_id"]
        urllib.urlretrieve(path_poster.format(extend_path), path_poster_local.format(IMDB_ID))

### 1.2 Download Data from IMDB

In [26]:
def download_imdb_n(id_list = "imdb_id25K.txt", out_file = "data_imdb_top100.txt", n = 100):
    ##### function downloads top n entires of imdb data as given by imdb_id25K.txt
    ##### outputs downloaded data as 

    # Import TMDb top 25,000 movies using imdb_ids
    ID = np.array(pd.read_csv(id_list, header=None))
    ID = ID[:n]

    # Among the different variables, we select the following variables of interest in our analysis  
    columns = ['title','genres', 'director', 'distributors', 'year', 'rating', 'votes', 'runtimes', 
              'language codes', 'languages', 'producer', 'mpaa', 'writer', 'top 250 rank', 'kind', 
               'country codes', 'countries', 'cover url', 'aspect_ratio', 'production companies', 
               'cinematographer', 'plot outline', 'plot', 'cast', 'animation department', 'original music',
               'canonical title', 'editorial department', 'canonical title', 'long imdb title',
               'long imdb canonical title', 'smart canonical title', 'smart long imdb canonical title',
               'full-size cover url']

    ## Download IMDb data
    index = range(1, len(ID)+1) 
    df = pd.DataFrame(index = index, columns=columns)
    df = df.fillna(0)

    # Fill in dataframe df 
    from imdb import IMDb
    ia = IMDb()
    for i in range(0, len(index)):
        movie = ia.get_movie(ID[i]) # grab movie data by id
        ia.update(movie)
        keys = movie.keys() # generate the available keys of this particular movie 
        #print i

        for j in range(0, len(columns)):
            if columns[j] in keys:
                if type(movie[columns[j]]) == list:
                    result = str(movie[columns[j]])
                else:
                    result = movie[columns[j]]
                df.iloc[i,j] = result
            else:
                df.iloc[i,j] = 'nan'

    # add column if IDs to the dataframe
    df['imdb_ids'] = pd.Series(ID.reshape(n,), index = df.index)

    # export dataframe to txt file
    df.to_csv(out_file, encoding='utf-8')

In [27]:
download_imdb_n(id_list = "imdb_id_demo.txt", out_file = "data_imdb_top100.txt", n = 100)

---
## Part2. Parse and Merge Data

For demenstration of code:
use the the top 100 rows

### 2.1 Parse IMDB Data

For the IMDB data (25K data points) we downloaded from IMDB server, 
we want to clean each column and extract the information.

- for response "genres", split multiple genres for one movie ans store them in a list
- for variables "runtimes", "language codes", "languages", "country codes", "countries": extract the multiple entries for one movie and store the result in a list
- for variables "director", "writer","cast","distributors", "producer", "production companies","cinematographer", "animation department", "original music", "editorial department": extract the multiple person IDs and company IDs for one movie and store the result in a list
- for variable "mpaa", we extract the MPAA rating for each movie, and store it under variable "mpaa"; we also keep the explaination of why the rating is made, and store the string under variable "mpaa_reason"

In [28]:
##### The function to parse data downloaded from IMDB
##### import the raw data by data_name, and specify the file name of the parsed data frame

def parse_imdb(data_name = "imdb_data.txt", new_name = "imdb_data_parse.txt"):

        import pandas as pd
        import numpy as np
        import re
        import warnings
        warnings.filterwarnings("ignore")

        #### load the imdb data to be parsed
        df = pd.read_csv(data_name)

        ## copy column "mpaa" to store the reason of MPAA rating
        df2 = df
        df2["mpaa_reason"] = df["mpaa"]

        
        #### select variables 
        #group0 = ["genres", "imdb_id"] 
        #group1 = ["title", "year", "rating", "votes", "top 250 rank", "kind"]
        #group2 = ["runtimes", "language codes", "languages", "country codes", "countries"]
        #group3 = ["mpaa", "mpaa_reason"]
        #group4 = ["director", "writer","cast","distributors", "producer", "production companies","cinematographer", "animation department", "original music", "editorial department"]
        
       
        
             
        #### for each row of the data
        for i in range(len(df)):

                ### -----------------------------------------------------------------------
                ## for variables with multiple items, return a list (later turn into dummy coding)

                for val in ["genres", "runtimes", "language codes", "languages", "country codes", "countries"]:
                        if type(df.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                                st = df.ix[i,val]
                                value = st[3:-2].split("', u'") 
                                df2[val][i] = value
  
                ### -----------------------------------------------------------------------
                ## extract movie MPAA rating as a string (later turn into factor)

                for val in ["mpaa"]:  
                        if type(df.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                            st = df.ix[i,val]
                            value = re.findall(r'Rated (.*?) for',st)[0] # extract the word between "Rated" and "for"
                            df2[val][i] = value
                ### -----------------------------------------------------------------------
                ## extract the reason of MPAA rating as a string (for text analysis later)

                for val in ["mpaa_reason"]:
                        if type(df.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                            st = df.ix[i,val]
                            value = st.split("for ", 1)[1]
                            df2[val][i] = value
                ### -----------------------------------------------------------------------
                ## for variables in the dictionary format, return a list of:
                ##     - person id 
                ##     - company id (for "production companies" and "distributors")

                for val in ["director", "writer","cast", 
                            "distributors", "producer", "production companies", 
                            "cinematographer", "animation department", 
                                "original music", "editorial department"]:
                        if type(df.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                            st = df.ix[i,val]
                            value = re.findall(r'\d+',st)
                            df2[val][i] = value
                            
                ### -----------------------------------------------------------------------
                ## variables that may not be useful:
                    # ["cover url", "full-size cover url"," canonical title", "canonical title.1", 
                    # "long imdb title", "long imdb canonical title", "smart canonical title", 
                    # "smart long imdb canonical title"]
                    
        
        df2.to_csv(new_name)


For demonstration purpose, here we use the top 100 rows of the database.

In [29]:
#parse_imdb(data_name = "imdb_top100_data.txt", new_name = "imdb_top100_data_parse.txt" )

In [30]:
imdb_top100_data_parse = pd.read_csv( "imdb_top100_data_parse.txt")
imdb_top100_data_parse.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1,title,genres,director,distributors,year,rating,votes,...,canonical title,editorial department,canonical title.1,long imdb title,long imdb canonical title,smart canonical title,smart long imdb canonical title,full-size cover url,imdb_ids,mpaa_reason
0,0,0,0,Four Rooms,['Comedy'],"['0025978', '0734319', '0001675', '0000233']","['0051881', '0014703', '0080682', '0022594', '...",1995.0,6.7,81543.0,...,Four Rooms,"['0020317', '0003692', '0042901', '0120656', '...",Four Rooms,Four Rooms (1995),Four Rooms (1995),Four Rooms,Four Rooms (1995),https://images-na.ssl-images-amazon.com/images...,113101,"pervasive strong language, sexuality and some ..."
1,1,1,1,"Sonntag, im August",['Short'],['1262754'],NaN,2004.0,6.5,11.0,...,"Sonntag, im August",NaN,"Sonntag, im August","Sonntag, im August (2004)","Sonntag, im August (2004)","Sonntag, im August","Sonntag, im August (2004)",https://images-na.ssl-images-amazon.com/images...,425473,NaN
2,2,2,2,Star Wars,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",['0000184'],"['0000756', '0074275', '20', '0105985', '20', ...",1977.0,8.7,969951.0,...,Star Wars,"['0091250', '0300879', '0366165', '0385492', '...",Star Wars,Star Wars (1977),Star Wars (1977),Star Wars,Star Wars (1977),https://images-na.ssl-images-amazon.com/images...,76759,sci-fi violence and brief mild language
3,3,3,3,Finding Nemo,"['Animation', 'Adventure', 'Comedy', 'Family']","['0004056', '0881279']","['0358618', '0064622', '0000779', '0110125', '...",2003.0,8.1,743705.0,...,Finding Nemo,"['0023224', '1404655', '1733701', '1404694', '...",Finding Nemo,Finding Nemo (2003),Finding Nemo (2003),Finding Nemo,Finding Nemo (2003),https://images-na.ssl-images-amazon.com/images...,266543,NaN
4,4,4,4,The Dark,"['Horror', 'Mystery', 'Thriller']",['0269502'],"['0002257', '0059064', '0022594', '0049085', '...",2005.0,5.4,9415.0,...,"Dark, The","['1262315', '3024002', '1930271', '0996933', '...","Dark, The",The Dark (2005),"Dark, The (2005)","Dark, The","Dark, The (2005)",https://images-na.ssl-images-amazon.com/images...,411267,some violent/disturbing images and language


### 2.2 Merge TMDB and IMDB

#### 2.2.1 Merge features from TMDB and IMDB

Primarily, we get all our data from IMDB and TMDB. We combine all information by API requests from these 2 sources. After combining, we remove duplicated features such as title, language codes, runtimes, country codes. Further, we deleted features which are non-informative, such as kind, cover url, poster_path, id, backdrop_path. IMDB_ID is also non-informative for prediction but we keep it as a common link between poster names and our dataset. Some other duplicated features are also removed, such as language codes (it is the same as language), country codes (same as countries) and so on.

After these, remaining features for prediction are: 'title', 'director', 'distributors', 'year', 'rating', 'votes', 'languages', 'producer', 'mpaa', 'writer', 'top 250 rank', 'countries', 'aspect_ratio', 'production companies', 'cinematographer', 'plot outline', 'plot', 'cast', 'animation department', 'original music', 'editorial department', 'imdb_ids', 'mpaa_reason', 'revenue_TMDB', 'overview_TMDB', 'tagline_TMDB', 'vote_count_TMDB', 'belongs_to_collection_TMDB', 'original_language_TMDB', 'status_TMDB',  'release_date_TMDB', 'popularity_TMDB', 'budget_TMDB', 'vote_average_TMDB' and 'runtime_TMDB'.
 

In [61]:
# import TMDB data
df_tmdb = pd.read_csv('TMDb_data_25K.csv',index_col=0)
df_tmdb

,poster_path,production_countries,revenue,overview,video,id,genres,title,tagline,vote_count,...,imdb_id,adult,backdrop_path,production_companies,release_date,popularity,original_title,budget,vote_average,runtime
0,/eQs5hh9rxrk1m4xHsIz1w11Ngqb.jpg,[u'United States of America'],4300000,It's Ted the Bellhop's first night on the job....,False,5,"[u'Crime', u'Comedy']",Four Rooms,Twelve outrageous guests. Four scandalous requ...,402,...,tt0113101,False,/3EqYpbGCE9S5GddU2K4cYzP5UmI.jpg,"[u'Miramax Films', u'A Band Apart']",12/25/1995,1.048212,Four Rooms,4000000,6.4,98.0
1,NaN,[u'Germany'],0,NaN,False,9,[u'Drama'],Sunday in August,NaN,1,...,tt0425473,False,NaN,[],9/2/2004,0.003213,Sonntag im August,0,1.0,15.0
2,/tvSlBzAdRE29bZe5yYWrJ2ds137.jpg,[u'United States of America'],775398007,Princess Leia is captured and held hostage by ...,False,11,"[u'Adventure', u'Action', u'Science Fiction']",Star Wars,"A long time ago in a galaxy far, far away...",5333,...,tt0076759,False,/4iJfYYoQzZcONB9hNzg0J0wWyPH.jpg,"[u'Lucasfilm', u'Twentieth Century Fox Film Co...",5/25/1977,10.412617,Star Wars,11000000,8.0,121.0
3,/zjqInUwldOBa0q07fOyohYCWxWX.jpg,[u'United States of America'],940335536,A tale which follows the comedic and eventful ...,False,12,"[u'Animation', u'Family']",Finding Nemo,"There are 3.7 trillion fish in the ocean, they...",4426,...,tt0266543,False,/n2vIGWw4ezslXjlP0VNxkp9wqwU.jpg,"[u'Walt Disney Pictures', u'Pixar Animation St...",5/30/2003,4.987594,Finding Nemo,94000000,7.5,100.0
4,/8fzjzQhLXl1afshhsE5Y3MGuco4.jpg,"[u'Germany', u'United Kingdom']",0,Ad??le and her daughter Sarah are traveling on...,False,17,"[u'Horror', u'Thriller', u'Mystery']",The Dark,One of the living for one of the dead.,61,...,tt0411267,False,/pkCC3myn5QuoLzplTlvZLB0w2AU.jpg,"[u'Constantin Film', u'Impact Pictures', u'Isl...",1/26/2006,0.347210,The Dark,0,5.5,87.0
5,/qriaeUUwdmlgethK3aSAx68mG05.jpg,[u'Germany'],650422,In a futuristic city sharply divided between t...,False,19,"[u'Drama', u'Science Fiction']",Metropolis,There can be no understanding between the hand...,552,...,tt0017136,False,/gmH0N0bdlYGN1EUr8U2cMQYyg78.jpg,[u'Universum Film (UFA)'],1/10/1927,1.558519,Metropolis,92620000,8.0,153.0
6,/vVj92VBFUBzjqzLvBiPVVlJDp78.jpg,"[u'Canada', u'Spain']",9726954,A Pedro Almodovar production in which a fatall...,False,20,"[u'Drama', u'Romance']",My Life Without Me,NaN,63,...,tt0314412,False,/caLMOD3ZJ3V7qjlxDXrzlJfBLgD.jpg,"[u'El Deseo', u'Milestone Productions']",3/7/2003,0.457017,My Life Without Me,0,7.1,106.0
7,/tkt9xR1kNX5R9rCebASKck44si2.jpg,[u'United States of America'],655011224,"Jack Sparrow, a freewheeling 17th-century pira...",False,22,"[u'Adventure', u'Fantasy', u'Action']",Pirates of the Caribbean: The Curse of the Bla...,Prepare to be blown out of the water.,5105,...,tt0325980,False,/8AUQ7YlJJA9C8kWk8P4YNHIcFDE.jpg,"[u'Walt Disney Pictures', u'Jerry Bruckheimer ...",7/9/2003,14.278019,Pirates of the Caribbean: The Curse of the Bla...,140000000,7.4,143.0
8,/vdENJAPObttowMtIwe9jgtbsEnq.jpg,[u'United States of America'],180949000,An assassin is shot at the altar by her ruthle...,False,24,"[u'Action', u'Crime']",Kill Bill: Vol. 1,Go for the kill.,3712,...,tt0266697,False,/kkS8PKa8c134vXsj2fQkNqOaCXU.jpg,"[u'Miramax Films', u'A Band Apart', u'Super Co...",10/10/2003,4.933327,Kill Bill: Vol. 1,30000000,7.6,111.0
9,/km6qw4qUkHQRN5y2U1juM1WkBFh.jpg,[u'United States of America'],159157447,"William Munny is a retired, once-ruthless kill...",False,33,[u'Western'],Unforgiven,Some legends will never be forgotten. Some wro...,881,...,tt0105695,False,/vwg6FGap02LkPSeWhiMLDoVwkAz.jpg,[u'Warner Bros.'],8/7/1992,2.385589,Unforgiven,14000000,7.7,131.0


In [62]:
# import iMDB data
df_imdb = pd.read_csv('imdb_top100_data.txt',index_col=0)
df_imdb

,Unnamed: 0.1,title,genres,director,distributors,year,rating,votes,runtimes,language codes,...,original music,canonical title,editorial department,canonical title.1,long imdb title,long imdb canonical title,smart canonical title,smart long imdb canonical title,full-size cover url,imdb_ids
0,0,Four Rooms,[u'Comedy'],"[<Person id:0025978[http] name:_Anders, Alliso...",[<Company id:0051881[http] name:_Concorde Film...,1995.0,6.7,81543.0,"[u'98', u'110::(workprint version)']",[u'en'],...,"[<Person id:0249375[http] name:_Edison, Combus...",Four Rooms,"[<Person id:0020317[http] name:_Allen, Bob_>, ...",Four Rooms,Four Rooms (1995),Four Rooms (1995),Four Rooms,Four Rooms (1995),https://images-na.ssl-images-amazon.com/images...,113101
1,1,"Sonntag, im August",[u'Short'],"[<Person id:1262754[http] name:_Meyer, Marc_>]",NaN,2004.0,6.5,11.0,[u'15'],[u'de'],...,"[<Person id:1909419[http] name:_Biegai, Christ...","Sonntag, im August",NaN,"Sonntag, im August","Sonntag, im August (2004)","Sonntag, im August (2004)","Sonntag, im August","Sonntag, im August (2004)",https://images-na.ssl-images-amazon.com/images...,425473
2,2,Star Wars,"[u'Action', u'Adventure', u'Fantasy', u'Sci-Fi']","[<Person id:0000184[http] name:_Lucas, George_>]",[<Company id:0000756[http] name:_Twentieth Cen...,1977.0,8.7,969951.0,"[u'121', u'125::(special edition)']",[u'en'],...,"[<Person id:0002354[http] name:_Williams, John_>]",Star Wars,"[<Person id:0091250[http] name:_Boekelheide, T...",Star Wars,Star Wars (1977),Star Wars (1977),Star Wars,Star Wars (1977),https://images-na.ssl-images-amazon.com/images...,76759
3,3,Finding Nemo,"[u'Animation', u'Adventure', u'Comedy', u'Fami...","[<Person id:0004056[http] name:_Stanton, Andre...","[<Company id:0358618[http] name:_BDG filmai_>,...",2003.0,8.1,743705.0,[u'100'],[u'en'],...,"[<Person id:0002353[http] name:_Newman, Thomas_>]",Finding Nemo,"[<Person id:0023224[http] name:_y Alvarez, Lui...",Finding Nemo,Finding Nemo (2003),Finding Nemo (2003),Finding Nemo,Finding Nemo (2003),https://images-na.ssl-images-amazon.com/images...,266543
4,4,The Dark,"[u'Horror', u'Mystery', u'Thriller']","[<Person id:0269502[http] name:_Fawcett, John_>]",[<Company id:0002257[http] name:_Constantin Fi...,2005.0,5.4,9415.0,"[u'93', u'Philippines:92::(cut)']","[u'en', u'cy']",...,"[<Person id:0125220[http] name:_Butt, Edmund_>]","Dark, The","[<Person id:1262315[http] name:_Alexander, Mar...","Dark, The",The Dark (2005),"Dark, The (2005)","Dark, The","Dark, The (2005)",https://images-na.ssl-images-amazon.com/images...,411267
5,5,Metropolis,"[u'Drama', u'Sci-Fi']","[<Person id:0000485[http] name:_Lang, Fritz_>]",[<Company id:0034406[http] name:_Paramount-Ufa...,1927.0,8.3,119193.0,"[u'153', u'Germany:210::(premiere cut)', u'Ger...",[u'de'],...,"[<Person id:0403288[http] name:_Huppertz, Gott...",Metropolis,"[<Person id:6258563[http] name:_Jones, Blake_>...",Metropolis,Metropolis (1927),Metropolis (1927),Metropolis,Metropolis (1927),https://images-na.ssl-images-amazon.com/images...,17136
6,6,My Life Without Me,"[u'Drama', u'Romance']","[<Person id:0170043[http] name:_Coixet, Isabel_>]",[<Company id:0059453[http] name:_A-Film Distri...,2003.0,7.6,21762.0,"[u'106', u'Argentina:107']",[u'en'],...,"[<Person id:0897350[http] name:_de Vilallonga,...",My Life Without Me,"[<Person id:0006971[http] name:_Blecua, Irene_...",My Life Without Me,My Life Without Me (2003),My Life Without Me (2003),My Life Without Me,My Life Without Me (2003),https://images-na.ssl-images-amazon.com/images...,314412
7,7,Pirates of the Caribbean: The Curse of the Bla...,"[u'Action', u'Adventure', u'Fantasy']","[<Person id:0893659[http] name:_Verbinski, Gor...",[<Company id:0044374[http] name:_Walt Disney C...,2003.0,8.0,853508.0,[u'143'],[u'en'],...,"[<Person id:0046004[http] name:_Badelt, Klaus_>]",Pirates of the Caribbean: The Curse of the Bla...,"[<Person id:0067659[http] name:_Belcher, Tim_>...",Pirates of the Caribbean: The Curse of the Bla...,Pirates of the Caribbean: The Curse

In [63]:
# drop duplicate or non-informative imdb features
imdb_column_drop = ['Unnamed: 0.1', 'Unnamed: 0.1', 'runtimes', 'cover url', 'language codes', 'kind', 'country codes', 'full-size cover url', 'canonical title', 'canonical title.1', 'long imdb title', 'long imdb canonical title', 'smart canonical title', 'smart long imdb canonical title', ]
df_imdb_dropped = df_imdb.drop(imdb_column_drop, axis=1)
df_imdb_dropped.head()

,title,genres,director,distributors,year,rating,votes,languages,producer,mpaa,...,aspect_ratio,production companies,cinematographer,plot outline,plot,cast,animation department,original music,editorial department,imdb_ids
0,Four Rooms,[u'Comedy'],"[<Person id:0025978[http] name:_Anders, Alliso...",[<Company id:0051881[http] name:_Concorde Film...,1995.0,6.7,81543.0,[u'English'],"[<Person id:0004744[http] name:_Bender, Lawren...","Rated R for pervasive strong language, sexuali...",...,NaN,"[<Company id:0022594[http] name:_Miramax_>, <C...","[<Person id:0006554[http] name:_García, Rodrig...",Four interlocking tales that take place in a f...,"[u""This movie features the collaborative direc...","[<Person id:0205423[http] name:_Davis, Sammi_>...","[<Person id:0177478[http] name:_Cooke, Pam_>, ...","[<Person id:0249375[http] name:_Edison, Combus...","[<Person id:0020317[http] name:_Allen, Bob_>, ...",113101
1,"Sonntag, im August",[u'Short'],"[<Person id:1262754[http] name:_Meyer, Marc_>]",NaN,2004.0,6.5,11.0,[u'German'],"[<Person id:1262754[http] name:_Meyer, Marc_>]",NaN,...,NaN,NaN,[<Person id:0689837[http] name:_Polsak-Lohmann...,A couple on a boat. Their love is burnt out. B...,[u'A couple on a boat. Their love is burnt out...,"[<Person id:0501854[http] name:_Lengyel, Rita_...",NaN,"[<Person id:1909419[http] name:_Biegai, Christ...",NaN,425473
2,Star Wars,"[u'Action', u'Adventure', u'Fantasy', u'Sci-Fi']","[<Person id:0000184[http] name:_Lucas, George_>]",[<Company id:0000756[http] name:_Twentieth Cen...,1977.0,8.7,969951.0,[u'English'],"[<Person id:0476030[http] name:_Kurtz, Gary_>,...",Rated PG for sci-fi violence and brief mild la...,...,NaN,"[<Company id:0071326[http] name:_Lucasfilm_>, ...","[<Person id:0852405[http] name:_Taylor, Gilber...",Luke Skywalker joins forces with a Jedi Knight...,"[u'The Imperial Forces, under orders from crue...","[<Person id:0000434[http] name:_Hamill, Mark_>...",NaN,"[<Person id:0002354[http] name:_Williams, John_>]","[<Person id:0091250[http] name:_Boekelheide, T...",76759
3,Finding Nemo,"[u'Animation', u'Adventure', u'Comedy', u'Fami...","[<Person id:0004056[http] name:_Stanton, Andre...","[<Company id:0358618[http] name:_BDG filmai_>,...",2003.0,8.1,743705.0,[u'English'],"[<Person id:1380146[http] name:_Gotoh, Jinko_>...",NaN,...,NaN,[<Company id:0017902[http] name:_Pixar Animati...,"[<Person id:0129269[http] name:_Calahan, Sharo...",After his son is captured in the Great Barrier...,"[u""A clown fish named Marlin lives in the Grea...","[<Person id:0000983[http] name:_Brooks, Albert...","[<Person id:1162887[http] name:_Baena, Carlos_...","[<Person id:0002353[http] name:_Newman, Thomas_>]","[<Person id:0023224[http] name:_y Alvarez, Lui...",266543
4,The Dark,"[u'Horror', u'Mystery', u'Thriller']","[<Person id:0269502[http] name:_Fawcett, John_>]",[<Company id:0002257[http] name:_Constantin Fi...,2005.0,5.4,9415.0,"[u'English', u'Welsh']","[<Person id:0027271[http] name:_Anderson, Paul...",Rated R for some violent/disturbing images and...,...,NaN,[<Company id:0002257[http] name:_Constantin Fi...,"[<Person id:0003791[http] name:_Sebaldt, Chris...",In mourning over the tragic drowning of their ...,"[u'While in Wales visiting her husband James, ...","[<Person id:0004742[http] name:_Bello, Maria_>...",NaN,"[<Person id:0125220[http] name:_Butt, Edmund_>]","[<Person id:1262315[http] name:_Alexander, Mar...",411267


In [64]:
# drop duplicate or non-informative imdb features
tmdb_column_drop = ['adult', 'poster_path', 'original_title', 'video', 'id', 'homepage', 'backdrop_path', 'production_companies', 'production_countries', 'spoken_languages', 'title']
df_tmdb_dropped = df_tmdb.drop(tmdb_column_drop, axis=1)
df_tmdb_dropped.head()

,revenue,overview,genres,tagline,vote_count,belongs_to_collection,original_language,status,imdb_id,release_date,popularity,budget,vote_average,runtime
0,4300000,It's Ted the Bellhop's first night on the job....,"[u'Crime', u'Comedy']",Twelve outrageous guests. Four scandalous requ...,402,0,en,Released,tt0113101,12/25/1995,1.048212,4000000,6.4,98.0
1,0,NaN,[u'Drama'],NaN,1,0,de,Released,tt0425473,9/2/2004,0.003213,0,1.0,15.0
2,775398007,Princess Leia is captured and held hostage by ...,"[u'Adventure', u'Action', u'Science Fiction']","A long time ago in a galaxy far, far away...",5333,{u'backdrop_path': u'/d8duYyyC9J5T825Hg7grmaab...,en,Released,tt0076759,5/25/1977,10.412617,11000000,8.0,121.0
3,940335536,A tale which follows the comedic and eventful ...,"[u'Animation', u'Family']","There are 3.7 trillion fish in the ocean, they...",4426,{u'backdrop_path': u'/2hC8HHRUvwRljYKIcQDMyMbL...,en,Released,tt0266543,5/30/2003,4.987594,94000000,7.5,100.0
4,0,Ad??le and her daughter Sarah are traveling on...,"[u'Horror', u'Thriller', u'Mystery']",One of the living for one of the dead.,61,0,en,Released,tt0411267,1/26/2006,0.347210,0,5.5,87.0


In [57]:
# rename columns in tmdb dataset
for i in range(len(df_tmdb_dropped.columns.values)):
    df_tmdb_dropped.columns.values[i] = df_tmdb_dropped.columns.values[i] + '_TMDB'
df_tmdb_dropped.head()

,revenue_TMDB,overview_TMDB,genres_TMDB,tagline_TMDB,vote_count_TMDB,belongs_to_collection_TMDB,original_language_TMDB,status_TMDB,imdb_id_TMDB,release_date_TMDB,popularity_TMDB,budget_TMDB,vote_average_TMDB,runtime_TMDB
0,4300000,It's Ted the Bellhop's first night on the job....,"[u'Crime', u'Comedy']",Twelve outrageous guests. Four scandalous requ...,402,0,en,Released,tt0113101,12/25/1995,1.048212,4000000,6.4,98.0
1,0,NaN,[u'Drama'],NaN,1,0,de,Released,tt0425473,9/2/2004,0.003213,0,1.0,15.0
2,775398007,Princess Leia is captured and held hostage by ...,"[u'Adventure', u'Action', u'Science Fiction']","A long time ago in a galaxy far, far away...",5333,{u'backdrop_path': u'/d8duYyyC9J5T825Hg7grmaab...,en,Released,tt0076759,5/25/1977,10.412617,11000000,8.0,121.0
3,940335536,A tale which follows the comedic and eventful ...,"[u'Animation', u'Family']","There are 3.7 trillion fish in the ocean, they...",4426,{u'backdrop_path': u'/2hC8HHRUvwRljYKIcQDMyMbL...,en,Released,tt0266543,5/30/2003,4.987594,94000000,7.5,100.0
4,0,Ad??le and her daughter Sarah are traveling on...,"[u'Horror', u'Thriller', u'Mystery']",One of the living for one of the dead.,61,0,en,Released,tt0411267,1/26/2006,0.347210,0,5.5,87.0


In [65]:
# extract all column names from imdb and tmdb datasets for merged dataset
columns = []
for i in range(len(df_imdb_dropped.columns.values)):
    columns += [(df_imdb_dropped.columns.values[i])]
for i in range(len(df_tmdb_dropped.columns.values)):
    columns += [(df_tmdb_dropped.columns.values[i])]

In [66]:
# create a dataframe to store information from both imdb and tmdb datasets
index = range(len(df_imdb_dropped)) 
df_merged = pd.DataFrame(index = index, columns=columns)
df_merged = df_merged.fillna(0)
df_merged.head()

,title,genres,director,distributors,year,rating,votes,languages,producer,mpaa,...,vote_count,belongs_to_collection,original_language,status,imdb_id,release_date,popularity,budget,vote_average,runtime
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
id_not_match_list = []
for i in range(len(df_imdb_dropped)):
    if i not in [5551, 12767, 15224, 24829]:   # these movies have trouble finding TMDB info, should remove later
        if str(df_imdb_dropped.imdb_ids[i]) in df_tmdb_dropped.imdb_id[i]:
            for j in range(len(df_imdb_dropped.columns.values)):
                df_merged.loc[i, df_imdb_dropped.columns.values[j]] = df_imdb_dropped.loc[i, df_imdb_dropped.columns.values[j]]
            for j in range(len(df_tmdb_dropped.columns.values)):
                df_merged.loc[i, df_tmdb_dropped.columns.values[j]] = df_tmdb_dropped.loc[i, df_tmdb_dropped.columns.values[j]]
        else:
            id_not_match_list += [i]

In [69]:
print "Number of IDs not match:", len(id_not_match_list)

Number of IDs not match: 0


#### 2.2.2 Merge genres from TMDB and IMDB

In [70]:
# function genre_assign
# input: genres for one movie, string type
# output: a string list, which are corresponding columns names for the genres of input movie 
def genre_assign(genre_temp):
    columns_list = ['Action', 'Adventure', 'Comedy', 'Crime', 'Fantasy', 'Family', 'Romance', 'Horror', 'Reality-TV', 'Western', 'Documentary', 'Biography', 'News', 'Film-Noir', 'Drama', 'Animation', 'Sci-Fi', 'Thriller', 'Short', 'Mystery', 'Sport', 'War', 'History', 'Game-Show', 'Music', 'Musical', 'Talk-Show', 'Foreign', 'TV movie']
    genre_list = []
    for i in columns_list:
        if i in genre_temp:
            genre_list += [i]
    if "Sci-Fi" not in genre_list:
        if "Science Fiction" in genre_temp:
            genre_list += ['Sci-Fi']
    return genre_list

In [71]:
# create a dataframe to store genres
columns = ['Action', 'Adventure', 'Comedy', 'Crime', 'Fantasy', 'Family', 'Romance', 'Horror', 'Reality-TV', 'Western', 'Documentary', 'Biography', 'News', 'Film-Noir', 'Drama', 'Animation', 'Sci-Fi', 'Thriller', 'Short', 'Mystery', 'Sport', 'War', 'History', 'Game-Show', 'Music', 'Musical', 'Talk-Show', 'Foreign', 'TV movie']
index = range(len(df_merged))
df_merge_y = pd.DataFrame(index = index, columns=columns)
df_merge_y = df_merge_y.fillna(0)

In [72]:
for i in range(len(df_merged)):
    genre_temp = str(df_tmdb.loc[i, "genres"]) + str(df_imdb.loc[i, "genres"])
    column_list_temp = genre_assign(genre_temp)
    for column in column_list_temp:
        df_merge_y.loc[i, column] = 1
df_merge_y.head()

,Action,Adventure,Comedy,Crime,Fantasy,Family,Romance,Horror,Reality-TV,Western,...,Mystery,Sport,War,History,Game-Show,Music,Musical,Talk-Show,Foreign,TV movie
0,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [26]:
# remove genres from merged dataframe
#df_merged = df_merged.drop(["genres", "genres_TMDB"], axis = 1)

In [23]:
# write the dataframe out
#df_merged.to_csv(('feature_multi_25K.csv'))
#df_merge_y.to_csv('genre_multi_25K.csv')

---
## Part3. Feature Engineering

For demenstration of code:
use variable `director` for the top 100 data

### 2.1 Variables regarding people/companies involved production (DONE)

#### 2.1.1 extract important top X IDs (for each genre)


#### Directors

In [73]:
## Function that takes in features csv and genres csv 
## and outputs list of top 5 directors associated with each genre

def directors_by_genre(features_csv ="feature_multi_top100.txt", genre_csv = "genre_multi_top100.txt"):
    import numpy as np 
    import pandas as pd
    import re
    import seaborn as sns 
    
    features = pd.read_csv(features_csv)
    genres = pd.read_csv(genre_csv)
    
    features2 = features
    features2["mpaa_reason"] = features["mpaa"]

    # parse director column into entries instead of str
    for i in range(len(features)):
        for val in ["director"]:
            if type(features.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                st = features.ix[i,val]
                value = re.findall(r'\d+',st)
                features2[val][i] = value
   
    director = features2['director']
        
    ##### list of directors simply by expanding out each of the 100 movie entries 
    director_list = []
    for i in range(len(director)):
        if director[i] > 0:
            for j in range(len(director[i])):
                director_list = np.append(director_list, director[i][j]) #list of directors (not unique)
    
    ##### dataframe of directors as index, genres as columns 
    director_genre = pd.DataFrame(index = director_list, columns = genres.columns)
    count = 0

    for i in range(len(director)):
        if director[i] > 0:
            for j in range(len(director[i])):
                director_genre.ix[count,] = genres.ix[i,]
                count = count + 1
    
    ##### dataframe of unique directors as index, genre as columns, counts for each genre as table content
    director_full = pd.DataFrame(index = np.unique(director_list), columns = genres.columns)

    for i in range(len(np.unique(director_list))):
        director_full.ix[i, ] = director_genre[director_genre.index == np.unique(director_list)[i]].sum(axis = 0)
    
    #### Clean dataframe
    director_full = director_full.drop('Unnamed: 0', 1) #remove first column 
    total_numbers_by_genre = director_genre.drop('Unnamed: 0', 1).sum(axis =0) #total number of each genre
    percentage = director_full/total_numbers_by_genre #director_full normalized by genre 
    
    #### Select the top five directors of each genre
    genre_director=pd.DataFrame(index = percentage.columns, columns = [1,2,3,4,5])
    for i in range(len(percentage.columns)):
        if percentage[percentage.columns[i]].sum() > 0:
            for j in range(0,5):
                genre_director.iloc[i,j] = percentage.sort(columns = percentage.columns[i], ascending=False).index.values[j]
    
    return(genre_director)

#### Distributors

In [74]:
## Function that takes in features csv and genres csv 
## and outputs list of top 5 distributors associated with each genre

def distributors_by_genre(features_csv ="feature_multi_top100.txt", genre_csv = "genre_multi_top100.txt"):
    import numpy as np 
    import pandas as pd
    import re
    import seaborn as sns 
    
    features = pd.read_csv(features_csv)
    genres = pd.read_csv(genre_csv)
    
    features2 = features
    features2["mpaa_reason"] = features["mpaa"]

    # parse director column into entries instead of str
    for i in range(len(features)):
        for val in ["distributors"]:
            if type(features.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                st = features.ix[i,val]
                value = re.findall(r'\d+',st)
                features2[val][i] = value
   
    distributors = features2['distributors']
        
    ##### list of distributors simply by expanding out each of the 100 movie entries 
    distributors_list = []
    for i in range(len(distributors)):
        if distributors[i] > 0:
            for j in range(len(distributors[i])):
                distributors_list = np.append(distributors_list, distributors[i][j]) #list of distributors (not unique)
    
    ##### dataframe of directors as index, genres as columns 
    distributors_genre = pd.DataFrame(index = distributors_list, columns = genres.columns)
    count = 0
    for i in range(len(distributors)):
        if distributors[i] > 0:
            for j in range(len(distributors[i])):
                distributors_genre.ix[count,] = genres.ix[i,]
                count = count + 1
    
    ##### dataframe of unique directors as index, genre as columns, counts for each genre as table content
    distributors_full = pd.DataFrame(index = np.unique(distributors_list), columns = genres.columns)
    for i in range(len(np.unique(distributors_list))):
        distributors_full.ix[i, ] = distributors_genre[distributors_genre.index == np.unique(distributors_list)[i]].sum(axis = 0)
    
    #### Clean dataframe
    distributors_full = distributors_full.drop('Unnamed: 0', 1) #remove first column 
    total_numbers_by_genre = distributors_genre.drop('Unnamed: 0', 1).sum(axis =0) #total number of each genre
    percentage = distributors_full/total_numbers_by_genre #director_full normalized by genre 
    
    #### Select the top five directors of each genre
    genre_distributors=pd.DataFrame(index = percentage.columns, columns = [1,2,3,4,5])
    for i in range(len(percentage.columns)):
        if percentage[percentage.columns[i]].sum() > 0:
            for j in range(0,5):
                genre_distributors.iloc[i,j] = percentage.sort(columns = percentage.columns[i], ascending=False).index.values[j]
    
    return(genre_distributors)

#### Cast

In [75]:
## Function that takes in features csv and genres csv 
## and outputs list of top 5 cast members associated with each genre

def cast_by_genre(features_csv ="feature_multi_top100.txt", genre_csv = "genre_multi_top100.txt"):
    import numpy as np 
    import pandas as pd
    import re
    import seaborn as sns 
    
    features = pd.read_csv(features_csv)
    genres = pd.read_csv(genre_csv)
    
    features2 = features
    features2["mpaa_reason"] = features["mpaa"]

    # parse director column into entries instead of str
    for i in range(len(features)):
        for val in ["cast"]:
            if type(features.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                st = features.ix[i,val]
                value = re.findall(r'\d+',st)
                features2[val][i] = value
   
    cast = features2['cast']
        
    ##### list of distributors simply by expanding out each of the 100 movie entries 
    cast_list = []
    for i in range(len(cast)):
        if cast[i] > 0:
            for j in range(len(cast[i])):
                cast_list = np.append(cast_list, cast[i][j]) #list of distributors (not unique)
    
    ##### dataframe of directors as index, genres as columns 
    cast_genre = pd.DataFrame(index = cast_list, columns = genres.columns)
    count = 0
    for i in range(len(cast)):
        if cast[i] > 0:
            for j in range(len(cast[i])):
                cast_genre.ix[count,] = genres.ix[i,]
                count = count + 1
    
    ##### dataframe of unique directors as index, genre as columns, counts for each genre as table content
    cast_full = pd.DataFrame(index = np.unique(cast_list), columns = genres.columns)
    for i in range(len(np.unique(cast_list))):
        cast_full.ix[i, ] = cast_genre[cast_genre.index == np.unique(cast_list)[i]].sum(axis = 0)
    
    #### Clean dataframe
    cast_full = cast_full.drop('Unnamed: 0', 1) #remove first column 
    total_numbers_by_genre = cast_genre.drop('Unnamed: 0', 1).sum(axis =0) #total number of each genre
    percentage = cast_full/total_numbers_by_genre #director_full normalized by genre 
    
    #### Select the top five directors of each genre
    genre_cast=pd.DataFrame(index = percentage.columns, columns = [1,2,3,4,5])
    for i in range(len(percentage.columns)):
        if percentage[percentage.columns[i]].sum() > 0:
            for j in range(0,5):
                genre_cast.iloc[i,j] = percentage.sort(columns = percentage.columns[i], ascending=False).index.values[j]
    
    return(genre_cast)

In [78]:
#directors_top5 = directors_by_genre(features_csv ="feature_multi_top100.csv", genre_csv = "genre_multi_top100.txt")
#distributors_top5 = distributors_by_genre(features_csv ="feature_multi_top100.csv", genre_csv = "genre_multi_top100.txt")
#cast_top5 = cast_by_genre(features_csv ="feature_multi_top100.csv", genre_csv = "genre_multi_top100.txt")

### 2.2 Variables for text analysis

For variables "title","plot outline", "plot", "mpaa_reason", "tagline_TMDB", we'd like to apply text analysis to extract the key words that might be informative for predicting genre.

To do this, we first perform routine bag-od-words analysis to filter out non-word items (puctuations and numbers) of the text, and turn the stirn into frequency counts of different worlds.

For each variable, we would then apply PCA to the matrix of bag-of-word, and keep the top PCs (the number of PCs to choose will be decided by the variance they can explain, but for now I'll choose 10 PCs as demonstration). This new matrix would become the new feature matrix for this specific variable. 

In [79]:
df = pd.read_csv("feature_multi_top100.csv")
df.shape

(100, 43)

In [80]:
col_words = []
colname = "plot outline"

df_col = df[colname]

In [81]:
import re
col_words = []

for i in range(len(df_col)):
    
    if type(df_col[i]) == str: 
        letters_only = re.sub("[^a-zA-Z]", " " , df_col[i])

        lower_case = letters_only.lower()  
        words = lower_case.split()

        meaningful_words = lower_case.split()
        words = ( " ".join(meaningful_words))
    
    else: words = "NA"

    col_words.append(words)

In [82]:
# run "pip install --user --install-option="--prefix=" -U scikit-learn" on terminal
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   
                                         tokenizer = None,    
                                         preprocessor = None, 
                                         stop_words = None,   
                                         max_features = 5000)

col_data = vectorizer.fit_transform(col_words)
col_data = pd.DataFrame(col_data.toarray())

vocab = vectorizer.get_feature_names()

In [83]:
df_new = pd.concat([df["imdb_ids"], col_data], axis = 1)
col_names = ["imdb_ids"] + vocab
df_new.columns = col_names

In [84]:
df_new.shape

(100, 1085)

We can see that the text analysis on the first 100 rows generate bag-of-words of size 1085. The next step is to apply PCA to reduce the feature dimension.

In [85]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
df_pca = pd.DataFrame(pca.fit_transform(col_data))

In [86]:
df_new_pca = pd.concat([df["imdb_ids"], df_pca], axis = 1)

col_names = ["imdb_ids"]
for i in range(10):
    i_name = "plot_outline_PC" + str(i)
    col_names.append(i_name)
    
df_new_pca.columns = col_names
df_new_pca.to_csv("plot_outline_text_analysis.txt")

In [87]:
df_new_pca = pd.read_csv("plot_outline_text_analysis.txt")
df_new_pca.head()

,Unnamed: 0,imdb_ids,plot_outline_PC0,plot_outline_PC1,plot_outline_PC2,plot_outline_PC3,plot_outline_PC4,plot_outline_PC5,plot_outline_PC6,plot_outline_PC7,plot_outline_PC8,plot_outline_PC9
0,0,113101,-1.315676,-0.935287,0.343145,0.383137,0.320858,-0.233861,-0.514925,0.053405,-0.719794,-0.200480
1,1,425473,-1.433995,-0.003525,-0.183151,0.772229,-0.808212,-0.052665,0.404954,-0.581529,0.118188,-0.319819
2,2,76759,1.662509,1.688691,0.334731,0.012857,-1.797232,-0.602420,-1.599357,-1.276661,1.096285,0.290487
3,3,266543,-0.489276,1.601552,0.522656,0.301160,0.023773,-0.457688,0.304046,0.460639,-0.378965,-0.282301
4,4,411267,-0.073206,-0.043791,0.744533,-0.365725,-0.670167,1.374429,0.374073,1.913573,1.001909,0.864914


## Other discusions to include

The exact form of X and Y depends on the ideas you had previously. In general though Y should involve the genre of a movie, and X the features you want to include to predict the genre. Remember from the lecture that more features does not necessarily equal better prediction performance. Use your application knowledge and the insight you gathered from your genre pair analysis and additional EDA to design Y. Do you want to include all genres? Are there genres that you assume to be easier to separate than others? Are there genres that could be grouped together? There is no one right answer here. We are looking for your insight, so be sure to describe your decision process in your notebook.

In preparation for the deep learning part we strongly encourage you to have two sets of training data X, one with the metadata and one with the movie posters. Make sure to have a common key, like the movie ID, to be able to link the two sets together. Also be mindful of the data rate when you obtain the posters. Time your requests and choose which poster resolution you need. In most cases w500 should be sufficient, and probably a lower resolution will be fine.

The notebook to submit this week should at least include:

Discussion about the imbalanced nature of the data and how you want to address it

Description of your data

What does your choice of Y look like?

Which features do you choose for X and why?

How do you sample your data, how many samples, and why?

Important: You do not need to upload the data itself to Canvas.

#### Discussion about the imbalanced nature of the data and how you want to address it

We notice that there is a imbalance in the amount of data available in each genre. For example, there are a lot of Action and Adventure movies, but very few moviews belonging to genres like Documentary, Music, etc..

To deal with this imbalancce of data, we plan to sample a large amount of movies in our training model among the  25,000 movies that we have downloaded from IMDb and TMDb. We can either sample down, by randomly sampling n samples from each genre, where n is the number of films in the genre with minimum number of films; or we can sample up, by randomly sampling m samples from each genre, where m is a large number, for genres that have movie entries fewer than m, we would use bootstrapping to sample from the available data. The value of m to use depends on the actual computation power of our computers, but should be as large as possible. We would start with a resonable number of m, for instance, 10,000, and increase the number if the model performs poorly. We should also note that we plan to combine some of the highly correlated genres into one meta-genre (e.g., Family and Animation), the distribution of the number of movies in each genre will change, and we will take this into consideration in carrying out sampling.

#### Description of your data

Description of your data (danqing)
We randomly sampled 25,000 movie entires from the TMDb database and used the correponding imdb_ids to extract information from the IMDb database for the same movies. We then merged the two database together and formed our own database with 25,000 entires of movies.

#### What does your choice of Y look like?

#### Q. What does your choice of Y look like?

##### 1. combine IMDB and TMDB
For now we take the union of genres from IMDB and TMDB databse. 
From the EDA in Milestone 1 we see that movies have around 1-7 genres, and in total there are 29 kinds of generes (we sampled 25k data points so our data should cover all the possible genres in the database).

So the raw response would be a vector of length 29, with binary coding of 0 or 1.


##### 2. group genres based on correlation matrix
The other form of Y we want to explore is to group certain genres to reduce the length of the response vector. 


From EDA in Milestone 1 we do see some strong correlations among genres. Below is the correlation matrix of 500 movies we sampled in Milestone 1:
< img src="corr plot_popular500.png">
The correlation matrix built from the 25k data would give us a even more accurate correlation.

We can then set a threshold for the correlation between different genres (for example: group `family` and `animation` since their correlation is above 0.8). We'll try several different threshold for grouping.

After grouping, the Y would be a shorter vector (the exact lengths depends on the threshold and needs tuning in cross validation) with binary coding.

#### Which features do you choose for X and why?

Primarily, we get all our data from IMDB and TMDB. We combine all information by API requests from these 2 sources. After combining, we remove duplicated features such as title, language codes, runtimes, country codes. Further, we deleted features which are non-informative, such as kind, cover url, poster_path, id, backdrop_path. IMDB_ID is also non-informative for prediction but we keep it as a common link between poster names and our dataset. Some other duplicated features are also removed, such as language codes (it is the same as language), country codes (same as countries) and so on.

After these, remaining features for prediction are: 'title', 'director', 'distributors', 'year', 'rating', 'votes', 'languages', 'producer', 'mpaa', 'writer', 'top 250 rank', 'countries', 'aspect_ratio', 'production companies', 'cinematographer', 'plot outline', 'plot', 'cast', 'animation department', 'original music', 'editorial department', 'imdb_ids', 'mpaa_reason', 'revenue_TMDB', 'overview_TMDB', 'tagline_TMDB', 'vote_count_TMDB', 'belongs_to_collection_TMDB', 'original_language_TMDB', 'status_TMDB',  'release_date_TMDB', 'popularity_TMDB', 'budget_TMDB', 'vote_average_TMDB' and 'runtime_TMDB'.
 
 We have shown in Milestone 1 that some of above selected features are strongly correlated with genres, such as 'year of release', 'ratings', 'votes' and so on. We haven't test some of the above features yet but we choose them based on our common sense.
 
One thing to note is that here we only excluded features we are sure duplicate or non-informative. We will further check remaining features when we model on our dataset.

#### How do you sample your data, how many samples, and why?

We get our data about movies from both IMDB and TMDB. There are about 2 million and 400,000 movies in IMDB and TMDB, respectively. However, these amount of data points is too many for us, both for storage and for computation. We don't find any pattern of IMDB IDs thus we don't have a clear clue to effectively sample a portion of IMDB IDs. However, for TMDB IDs, it's sequential, so we can randomly pick some numbers (we used 25K here) in the range of 0 to the largest number. After getting 25K random TMDB IDs, we can query each of them and get information about each movie, including corresponding IMDB IDs. Thus we can query IMDB IDs and combine information of these 2 datasets.

We sampled 25K samples because we think this number would be enough for all our prediction methods but not too many to be a computational burden. However, we are open to any necessary changes to this number, if this amount is not enough, we can get more data points. If it takes too much time to training a model, we can randomly delete some of these movies.

If this number becomes a computational burden, we can also try to sample data points from these 25K dataset to test and optimize our models. For this purpose, we can start with sampling maybe 1000, do tests and decide if we should or if we can increase this number.

Another perspective of sampling is to tackle imbalance nature of the genres. We can sample out equal amount of movies in each genre for our training and testing datasets. We think 500 movies should be enough for each genre.
